In [ ]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')

In [ ]:
distances = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
distances.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
distances.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
distances

In [ ]:
lats = []
lons = []
for i in distances['Postal Code']:
    location = nomi.query_postal_code(i)
    lats.append(location['latitude'])
    lons.append(location['longitude'])
distances['lat'] = lats
distances['lon'] = lons
distances['geometry'] = geopandas.points_from_xy(distances['lat'], distances['lon'])
distances.sort_values(by='lat', inplace=True)
distances

In [ ]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = distances['lat'],
    lon = distances['lon'],
    text = distances['Member']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig0.show()
#fig.write_html('membermap.html')

In [ ]:
#!pip install kaleido
#fig0.write_image('membermap.svg')

## Grouping dots together

In [ ]:
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs

# Calculate the distance between each member
distances_list = []
for i in range(len(distances)):
    for j in range(i+1, len(distances)):
        distances_list.append([distances.iloc[i]['Member'], distances.iloc[j]['Member'], 
                          hs.haversine((distances.iloc[i]['lat'], distances.iloc[i]['lon']), 
                                       (distances.iloc[j]['lat'], distances.iloc[j]['lon']))])
distances_between = pd.DataFrame(distances_list, columns=['Member', 'Member2', 'Distance'])

# join the latitude and longitude of each member to the distances
distances_between = distances_between.merge(distances[['Member', 'lat', 'lon']], left_on='Member', right_on='Member', how='left')
distances_between

In [ ]:
# for each member create a list of the other members that are within 100 km

def findNeighbours(m):
    return distances_between[(distances_between['Member'] == m) & (distances_between['Distance'] < 100)]['Member2'].values

distances_between['Neighbours'] = distances_between['Member'].apply(findNeighbours)
distances_between

In [ ]:
#asdf = distances_between[distances_between['Distance'] < 15]
for x in range(0, 500, 5):
    asdf = distances_between[distances_between['Distance'] < x]
    print(x, len(asdf))

In [ ]:
#distances_between.to_csv('data/distances_between.csv', index=False)

In [ ]:
# this will only work if they are sorted, but it's not great
# we should elimilate one of the two members in a pair that is too close
# but there are lots of groups that are three or more members close together

separation_distance = 150

previous_row = None
rows_to_remove = []
for row in distances.sort_values(by='lat').itertuples():
    #print(row.Member, row.lat, row.lon)
    if previous_row is not None:
        #print(previous_row.Member, row.Member, 
        d = hs.haversine((previous_row.lat, previous_row.lon), (row.lat, row.lon))
        if d < separation_distance:
            #print('remove row', row.Index)
            rows_to_remove.append(row.Index)
    previous_row = row
filtered_distances = distances.drop(rows_to_remove, axis=0)
import folium
#from folium.plugins import FastMarkerCluster
median_latitude = filtered_distances['lat'].median()
median_longitude = filtered_distances['lon'].median()
new_map = folium.Map(location=[median_latitude, median_longitude], zoom_start=6, tiles='stamen terrain')
#new_map.add_child(FastMarkerCluster(filtered_distances[['lat','lon']].values.tolist()))
for i in filtered_distances.itertuples():
    folium.Marker([i.lat, i.lon], popup=i.Member).add_to(new_map)
display(new_map)

In [ ]:
len(distances_list[distances_list['Distance'] < 20].sort_values('Distance')['Member1'].unique())

In [ ]:
# map the members that are close to each other
distances_list['Member 1 Lat'] = distances_list['Member1'].map(distances_list.set_index('Member')['lat'])
distances_list['Member 1 Lon'] = distances_list['Member1'].map(distances_list.set_index('Member')['lon'])
distances_list

In [ ]:
distances_list[(distances_list['Member1']=='Alberta Conference SDA Church') & (distances_list['Member2']=='Alberta Conference SDA Church')]

In [ ]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = distances_list['Member 1 Lat'],
    lon = distances_list['Member 1 Lon'],
    text = distances_list['Member1']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig0.show()

## Finding scale distances on a 4x8 sheet of plywood

In [ ]:
x_range = [distances_list['lon'].max(), distances_list['lon'].min()]
y_range = [distances_list['lat'].max(), distances_list['lat'].min()]
distances_list['x'] = (distances_list['lon'] - x_range[1]) / (x_range[0] - x_range[1])
distances_list['y'] = (distances_list['lat'] - y_range[1]) / (y_range[0] - y_range[1])
distances_list['x inches'] = distances_list['x'] * 46
distances_list['y inches'] = distances_list['y'] * 46
distances_list

In [ ]:
import plotly.express as px
fig2 = px.scatter(distances_list, x='x inches', y='y inches', hover_name='Member', hover_data=['Address', 'City', 'Postal Code'], height=800, width=400)
fig2.update_xaxes(range=[0, 48])
fig2.update_yaxes(range=[0, 96])
fig2.show()
fig0.show()